In [1]:
%pip install tensorflow==2.5.0 scikit-learn optuna

ERROR: Could not find a version that satisfies the requirement tensorflow==2.5.0 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0)
ERROR: No matching distribution found for tensorflow==2.5.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np

In [3]:
def dummy_npwarn_decorator_factory():
  def npwarn_decorator(x):
    return x
  return npwarn_decorator
np._no_nep50_warning = getattr(np, '_no_nep50_warning', dummy_npwarn_decorator_factory)

In [4]:
import tensorflow as tf

2025-02-21 12:09:11.771131: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-21 12:09:11.852918: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-21 12:09:11.921788: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740119951.986682    5095 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740119952.005153    5095 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-21 12:09:12.163998: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [5]:
print(tf.__version__)
print(np.__version__)

2.18.0
2.0.2


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [7]:
def get_mlp_model(input_shape,hidden_layer_one,dropout_one,hidden_layer_two,dropout_two):
    model = Sequential()
    model.add(Dense(hidden_layer_one, activation='elu', input_shape=input_shape))
    if dropout_one !=0:
        model.add(Dropout(dropout_one))
    model.add(Dense(hidden_layer_two, activation='elu'))
    if dropout_two !=0:
        model.add(Dropout(dropout_two))
    model.add(Dense(1))
    # Compile the model
    model.compile(loss='mean_squared_error', optimizer="Adam")

    return model

In [8]:
def compute_error(trues, predicted):
    corr = np.corrcoef(predicted, trues)[0,1]
    mae = np.mean(np.abs(predicted - trues))
    rae = np.sum(np.abs(predicted - trues)) / np.sum(np.abs(trues - np.mean(trues)))
    rmse = np.sqrt(np.mean((predicted - trues)**2))
    rrse = np.sqrt(np.sum((predicted - trues)**2) / np.sum((trues - np.mean(trues))**2))
    mape = np.mean(np.abs((predicted - trues) / trues)) * 100
    r2 = max(0, 1 - np.sum((predicted - trues)**2) / np.sum((trues - np.mean(trues))**2))
    # Calculez les autres mesures d'erreur ici
    return rmse, corr, mae, rae, rrse, mape, r2

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import optuna
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.neural_network import MLPRegressor
from tensorflow.keras.callbacks import ModelCheckpoint

/home/usman/miniconda3/envs/curie/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

W0000 00:00:1740119954.938906    5095 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


[]

# We start by loading the pre-processed data

In [11]:
order_history_data=pd.read_csv('../data/final_data.csv')
order_history_data.head()

,Inflation,Month,Year,pmi,PartNo,Description,Quantity,Sales Price,Total Sales,Swaraj Engines Stock Price,M&M Stock Price,Escorts Kuboto Stock Price
0,7.59,1,2020,250,0503BA0290N-RK,ALFA HUB,85,2118.60,59367,1125.0,532.9,630.0
1,7.59,1,2020,250,0503BA0290N,Front Wheel Hub,318,3536.35,59367,1125.0,532.9,630.0
2,7.59,1,2020,250,0502BA1050N,Alfa Rear Wheel Hub LH,209,1383.36,59367,1125.0,532.9,630.0
3,7.59,1,2020,250,0502BA1060N,Alfa Rear Wheel Hub RH,119,4849.50,59367,1125.0,532.9,630.0
4,7.59,1,2020,250,0602AAB00260N,Brake Drum ALFA,134,2462.26,59367,1125.0,532.9,630.0


# Deep learning model optimization

In [12]:
X = order_history_data.copy()
y = order_history_data['Quantity']

X_encoded = pd.get_dummies(X, columns=['Description', 'PartNo'])

X_train, X_temp, y_train, y_temp = train_test_split(X_encoded, y, test_size=0.3, random_state=27)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=27)

We're going to use mlp. To start with, we'll create an mlp with minimal parameters to get an idea of performance without model optimization. Then we'll optmize our model using Bayesian optimization.

In [13]:

best_mlp = MLPRegressor(hidden_layer_sizes=(50), activation='relu', alpha=0.008724528119026307, learning_rate='constant')

# Train the model on the training data
best_mlp.fit(X_train, y_train)

# Make predictions on the validation set with the trained model
y_val_pred = best_mlp.predict(X_val)

# Evaluate the performance of the model on the validation set
mse_val = mean_squared_error(y_val, y_val_pred)
r2_val = best_mlp.score(X_val, y_val)

# Make predictions on the test set with the trained model
y_test_pred = best_mlp.predict(X_test)

# Evaluate the performance of the model on the test set
mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = best_mlp.score(X_test, y_test)

print("Validation Set:")
print("MSE (Mean Squared Error) on the validation set:", mse_val)
print("R-squared (Coefficient of Determination) on the validation set:", r2_val)

print("\nTest Set:")
print("MSE (Mean Squared Error) on the test set:", mse_test)
print("R-squared (Coefficient of Determination) on the test set:", r2_test)



Validation Set:
MSE (Mean Squared Error) on the validation set: 5.612080751813463
R-squared (Coefficient of Determination) on the validation set: 0.9997392727056681

Test Set:
MSE (Mean Squared Error) on the test set: 15.315567046765025
R-squared (Coefficient of Determination) on the test set: 0.9993183575334714


Using Bayesian optimization to find the best hyperparameters

In [14]:

# Objective function for Optuna optimization
def objective(trial):
    # Define the hyperparameters to be optimized
    hidden_layer_one = trial.suggest_categorical('hidden_layer_one', [32, 64, 128, 256,512])
    hidden_layer_two = trial.suggest_categorical('hidden_layer_two', [32, 64, 128, 256,512])
    dropout_one = trial.suggest_uniform('dropout_one', 0, 0.5)
    dropout_two = trial.suggest_uniform('dropout_two', 0, 0.5)
    # learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128,256,512])
    epochs = trial.suggest_categorical('epochs', [200])



    model=get_mlp_model(input_shape=(X_train.shape[1],),hidden_layer_one=hidden_layer_one,dropout_one=dropout_one,hidden_layer_two=hidden_layer_two,dropout_two=dropout_two)


    # Define the ModelCheckpoint callback
    checkpoint_filepath = 'model_checkpoint.h5'
    model_checkpoint = ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_best_only=True,
        monitor='val_loss',
        mode='min',
        verbose=0
    )

    # Train the model on the training data with validation data and checkpoint callback
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0
                        , validation_data=(X_val, y_val),
                        callbacks=[model_checkpoint])

    # Load the best weights from the saved checkpoint
    model.load_weights(checkpoint_filepath)

    # Evaluate the performance of the model on the test set
    y_test_pred = model.predict(X_test)

    # Calculate performance metrics
    mse_test = mean_squared_error(y_test, y_test_pred)
    # rmse2, corr2, mae2, rae2, rrse2, mape2, r2_2 = compute_error(y_test.values, y_test_pred.reshape(y_test_pred.shape[0]))

    return mse_test

# Configure Optuna to use the GPU for exhaustive searches
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100, n_jobs=1)  # Use n_jobs=1 to avoid parallelism problems on GPU

# Show best hyperparameters found
print("Best hyperparameters:")
print(study.best_params)
print("Best MSE:", study.best_value)


[I 2025-02-21 12:09:17,187] A new study created in memory with name: no-name-3d329e0e-e18c-4a05-8b03-649c3bc0905d
/tmp/ipykernel_5095/3352955203.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_one = trial.suggest_uniform('dropout_one', 0, 0.5)
/tmp/ipykernel_5095/3352955203.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_two = trial.suggest_uniform('dropout_two', 0, 0.5)
/home/usman/miniconda3/envs/curie/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regula

49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


[I 2025-02-21 12:09:40,022] Trial 0 finished with value: 1720.8018798828125 and parameters: {'hidden_layer_one': 32, 'hidden_layer_two': 128, 'dropout_one': 0.22201321274675928, 'dropout_two': 0.46620758805654866, 'batch_size': 128, 'epochs': 200}. Best is trial 0 with value: 1720.8018798828125.
/tmp/ipykernel_5095/3352955203.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_one = trial.suggest_uniform('dropout_one', 0, 0.5)
/tmp/ipykernel_5095/3352955203.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_two = trial.suggest_uniform('dropout_two', 0, 0.5)
/home/usman/miniconda3/envs/curie/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_

49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


[I 2025-02-21 12:10:34,657] Trial 1 finished with value: 18.174707412719727 and parameters: {'hidden_layer_one': 32, 'hidden_layer_two': 512, 'dropout_one': 0.041900613477572624, 'dropout_two': 0.2187841965219503, 'batch_size': 32, 'epochs': 200}. Best is trial 1 with value: 18.174707412719727.
/tmp/ipykernel_5095/3352955203.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_one = trial.suggest_uniform('dropout_one', 0, 0.5)
/tmp/ipykernel_5095/3352955203.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_two = trial.suggest_uniform('dropout_two', 0, 0.5)
/home/usman/miniconda3/envs/curie/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_s

...

49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


[I 2025-02-21 13:27:09,796] Trial 98 finished with value: 56.75661849975586 and parameters: {'hidden_layer_one': 64, 'hidden_layer_two': 512, 'dropout_one': 0.047815099214071066, 'dropout_two': 0.2563464716798771, 'batch_size': 256, 'epochs': 200}. Best is trial 11 with value: 1.6968963146209717.
/tmp/ipykernel_5095/3352955203.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_one = trial.suggest_uniform('dropout_one', 0, 0.5)
/tmp/ipykernel_5095/3352955203.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_two = trial.suggest_uniform('dropout_two', 0, 0.5)
/home/usman/miniconda3/envs/curie/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input

49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


[I 2025-02-21 13:27:49,131] Trial 99 finished with value: 11.53165340423584 and parameters: {'hidden_layer_one': 64, 'hidden_layer_two': 512, 'dropout_one': 0.010687227549120389, 'dropout_two': 0.4262221927008326, 'batch_size': 64, 'epochs': 200}. Best is trial 11 with value: 1.6968963146209717.


Best hyperparameters:
{'hidden_layer_one': 64, 'hidden_layer_two': 512, 'dropout_one': 0.0018129937342037036, 'dropout_two': 0.17694983148756732, 'batch_size': 16, 'epochs': 200}
Best MSE: 1.6968963146209717


Using Grid Search

Training model with best hyperparamters

In [15]:
# best parameters

hidden_layer_one=50
hidden_layer_two=100
dropout_one=0
dropout_two=0


model=get_mlp_model(input_shape=(X_train.shape[1],),hidden_layer_one=hidden_layer_one,dropout_one=dropout_one,hidden_layer_two=hidden_layer_two,dropout_two=dropout_two)

# Define the ModelCheckpoint callback
checkpoint_filepath = 'model_checkpoint.h5'
model_checkpoint = ModelCheckpoint(
filepath=checkpoint_filepath,
save_best_only=True,
monitor='val_loss',
mode='min',
verbose=0
)

# Train the model on the training data with validation data and checkpoint callback
history = model.fit(X_train, y_train, epochs=300, batch_size=512,
            verbose=2, validation_data=(X_val, y_val),
            callbacks=[model_checkpoint])

# Load the best weights from the saved checkpoint
model.load_weights(checkpoint_filepath)

# Evaluate the performance of the model on the test set
y_test_pred = model.predict(X_test)

# Calculate performance metrics
rmse2, corr2, mae2, rae2, rrse2, mape2, r2_2 = compute_error(y_test.values, y_test_pred.reshape(y_test_pred.shape[0]))

print("RMSE:", rmse2)
print("Corrélation:", corr2)
print("MAE:", mae2)
print("RAE:", rae2)
print("RRSE:", rrse2)
print("MAPE:", mape2)
print("R2:", r2_2)
print("----------------------------------")

Epoch 1/300


/home/usman/miniconda3/envs/curie/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


15/15 - 1s - 44ms/step - loss: 2146706.7500 - val_loss: 959487.8125
Epoch 2/300


15/15 - 0s - 6ms/step - loss: 353605.4062 - val_loss: 31386.7656
Epoch 3/300
15/15 - 0s - 5ms/step - loss: 71014.5391 - val_loss: 65177.5859
Epoch 4/300


15/15 - 0s - 5ms/step - loss: 35087.3945 - val_loss: 28514.7969
Epoch 5/300


...

15/15 - 0s - 5ms/step - loss: 11.1149 - val_loss: 4.0687
Epoch 106/300
15/15 - 0s - 5ms/step - loss: 38.2846 - val_loss: 64.5039
Epoch 107/300
15/15 - 0s - 5ms/step - loss: 26.6758 - val_loss: 5.6579
Epoch 108/300
15/15 - 0s - 5ms/step - loss: 19.2498 - val_loss: 5.7293
Epoch 109/300
15/15 - 0s - 5ms/step - loss: 24.8092 - val_loss: 72.2989
Epoch 110/300
15/15 - 0s - 4ms/step - loss: 43.9300 - val_loss: 7.6588
Epoch 111/300
15/15 - 0s - 5ms/step - loss: 43.1732 - val_loss: 7.4882
Epoch 112/300
15/15 - 0s - 5ms/step - loss: 65.2993 - val_loss: 44.3958
Epoch 113/300
15/15 - 0s - 5ms/step - loss: 16.4167 - val_loss: 7.5473
Epoch 114/300


15/15 - 0s - 5ms/step - loss: 10.0611 - val_loss: 2.6210
Epoch 115/300
15/15 - 0s - 5ms/step - loss: 4.4473 - val_loss: 20.6839
Epoch 116/300
15/15 - 0s - 5ms/step - loss: 67.2567 - val_loss: 58.9975
Epoch 117/300
15/15 - 0s - 5ms/step - loss: 104.5743 - val_loss: 21.3569
Epoch 118/300
15/15 - 0s - 5ms/step - loss: 28.9933 - val_loss: 63.0358
Epoch 119/300
15/15 - 0s - 5ms/step - loss: 34.0303 - val_loss: 4.2929
Epoch 120/300
15/15 - 0s - 5ms/step - loss: 112.2888 - val_loss: 329.9616
Epoch 121/300
15/15 - 0s - 5ms/step - loss: 173.0649 - val_loss: 10.3162
Epoch 122/300
15/15 - 0s - 5ms/step - loss: 58.1648 - val_loss: 32.5285
Epoch 123/300
15/15 - 0s - 5ms/step - loss: 34.7394 - val_loss: 33.0751
Epoch 124/300
15/15 - 0s - 5ms/step - loss: 27.0249 - val_loss: 7.5055
Epoch 125/300


15/15 - 0s - 5ms/step - loss: 6.6531 - val_loss: 2.0061
Epoch 126/300
15/15 - 0s - 5ms/step - loss: 2.6689 - val_loss: 2.0826
Epoch 127/300
15/15 - 0s - 5ms/step - loss: 4.9579 - val_loss: 4.7370
Epoch 128/300
15/15 - 0s - 4ms/step - loss: 12.2825 - val_loss: 7.1714
Epoch 129/300
15/15 - 0s - 4ms/step - loss: 17.6807 - val_loss: 2.9533
Epoch 130/300
15/15 - 0s - 5ms/step - loss: 315.1968 - val_loss: 967.4889
Epoch 131/300
15/15 - 0s - 4ms/step - loss: 340.6017 - val_loss: 313.9178
Epoch 132/300
15/15 - 0s - 4ms/step - loss: 129.4363 - val_loss: 62.2965
Epoch 133/300
15/15 - 0s - 5ms/step - loss: 33.2143 - val_loss: 127.4470
Epoch 134/300
15/15 - 0s - 5ms/step - loss: 69.7549 - val_loss: 6.3469
Epoch 135/300
15/15 - 0s - 5ms/step - loss: 60.9730 - val_loss: 64.1859
Epoch 136/300
15/15 - 0s - 4ms/step - loss: 20.7863 - val_loss: 4.2163
Epoch 137/300
15/15 - 0s - 5ms/step - loss: 16.6606 - val_loss: 32.4806
Epoch 138/300
15/15 - 0s - 5ms/step - loss: 21.1025 - val_loss: 10.8973
Epoch 139/

15/15 - 0s - 5ms/step - loss: 6.7710 - val_loss: 1.6887
Epoch 142/300
15/15 - 0s - 5ms/step - loss: 12.1944 - val_loss: 2.0347
Epoch 143/300
15/15 - 0s - 5ms/step - loss: 9.2630 - val_loss: 7.9636
Epoch 144/300
15/15 - 0s - 5ms/step - loss: 67.1181 - val_loss: 238.4173
Epoch 145/300
15/15 - 0s - 5ms/step - loss: 184.1492 - val_loss: 361.7310
Epoch 146/300
15/15 - 0s - 5ms/step - loss: 127.3447 - val_loss: 55.3148
Epoch 147/300
15/15 - 0s - 5ms/step - loss: 35.2889 - val_loss: 25.3130
Epoch 148/300
15/15 - 0s - 5ms/step - loss: 30.0379 - val_loss: 5.4482
Epoch 149/300


15/15 - 0s - 5ms/step - loss: 9.8385 - val_loss: 1.6122
Epoch 150/300
15/15 - 0s - 5ms/step - loss: 12.8515 - val_loss: 20.3431
Epoch 151/300
15/15 - 0s - 5ms/step - loss: 190.2620 - val_loss: 2.4730
Epoch 152/300
15/15 - 0s - 5ms/step - loss: 2156.1729 - val_loss: 2332.4375
Epoch 153/300
15/15 - 0s - 5ms/step - loss: 1142.3643 - val_loss: 206.9333
Epoch 154/300
15/15 - 0s - 5ms/step - loss: 443.0380 - val_loss: 922.2822
Epoch 155/300
15/15 - 0s - 5ms/step - loss: 376.7956 - val_loss: 43.7223
Epoch 156/300
15/15 - 0s - 5ms/step - loss: 88.9247 - val_loss: 42.8968
Epoch 157/300
15/15 - 0s - 5ms/step - loss: 32.3679 - val_loss: 28.6330
Epoch 158/300
15/15 - 0s - 5ms/step - loss: 26.4860 - val_loss: 39.6797
Epoch 159/300
15/15 - 0s - 5ms/step - loss: 25.9110 - val_loss: 26.9525
Epoch 160/300
15/15 - 0s - 5ms/step - loss: 17.2006 - val_loss: 26.3147
Epoch 161/300
15/15 - 0s - 4ms/step - loss: 14.3496 - val_loss: 10.4492
Epoch 162/300
15/15 - 0s - 4ms/step - loss: 10.9715 - val_loss: 10.909

Save the weights of the best model for later loading

In [19]:
# Save the best model in checkpoint only if it is better
model.save_weights('../checkpoint/base_model_checkpoint.weights.h5')